# 보상 파일 불러오기

연도는 이미 열에 존재하기 때문에 모든 데이터 병합

In [1]:
import pandas as pd

file_path = '../data/Compensation.xlsx'
sheet_names = [str(year) for year in range(2020,2025)]

dfs = [pd.read_excel(file_path, sheet_name = sheet) for sheet in sheet_names]

merged_df = pd.concat(dfs, ignore_index = True)

In [2]:
merged_df

,구분,지역,교육청,학교급,사고자구분,사고자학년,사고자성별,사고시간,사고장소,사고부위,사고형태,사고당시활동,요양급여,장해급여,간병급여,유족급여,장례비,위로금,보전비용
0,F0000001,전남,전라남도교육청,고등학교,일반학생,2학년,남,쉬는시간,강당(체육관),골반/엉덩이,그밖의 손상 사고,기타,31900,0,0,0,0,0,0
1,F0000002,대구,남부교육지원청,초등학교,일반학생,6학년,여,체육,운동장,두피,그밖의 손상 사고,기타 스포츠 활동,53000,0,0,0,0,0,0
2,F0000003,경남,경상남도교육청,고등학교,일반학생,1학년,남,그 밖의 교육활동 시간,기타 교외,기타,그밖의 손상 사고,기타,0,0,15027050,0,0,0,0
3,F0000004,전북,군산교육지원청,초등학교,일반학생,5학년,남,쉬는시간,일반(교과)교실,어깨,그밖의 손상 사고,기타,3114900,0,0,0,0,0,0
4,F0000005,제주,제주시교육지원청,초등학교,일반학생,2학년,남,식사시간(간식 포함),기타 교외,치아,고정된 물체와의 부딪힘,기타,1106500,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425209,K0132478,서울,성북강북교육지원청,중학교,일반학생,3학년,여,체육,강당(체육관),손가락,고정된 물체와의 부딪힘,피구,97200,0,0,0,0,0,0
425210,K0132479,서울,강동송파교육지원청,초등학교,일반학생,5학년,남,체육,강당(체육관),복합부위,사람과의 부딪힘,기타 구기,585950,0,0,0,0,0,0
425211,K0132480,서울,서울특별시교육청,기타학교,일반학생,3학년,여,현장학습,기타 교외,무릎,넘어짐,"걷기/뛰기, 오르내리기",0,0,0,0,0,0,0
425212,K0132481,서울,서울특별시교육청,기타학교,일반학생,2학년,여,쉬는시간,계단,어깨,넘어짐,"걷기/뛰기, 오르내리기",284910,0,0,0,0,0,0


# 전처리

- 사고 데이터와 같은 처리
    - na행 제거
      - 존재하는 na는 교직원의 사고로 학교안전사고의 대부분인 학생 위주 분석하기 위함
    - 불필요 feature 제거
      - 안전교육홍보 목적: 일반화를 위해 지역을 남기고 교육청은 제거
    - 파생변수 생성
      - 학교급과 학년을 조합하여 일반적인 나이 열을 생성: 발달적인 부분을 보기 위함
     
- 보상 데이터의 보상 항목
    - 유형이 다르지만 일반적으로 보상 금액이 높을 수록 사고 수준도 클것이라고 판단
    - 유형의 금액을 단순 보상비용으로 합쳐 확인

In [5]:
merged_df.isna().sum()

구분        0
지역        0
교육청       0
학교급       0
사고자구분     0
사고자학년     0
사고자성별     0
사고시간      0
사고장소      0
사고부위      0
사고형태      0
사고당시활동    0
요양급여      0
장해급여      0
간병급여      0
유족급여      0
장례비       0
위로금       0
보전비용      0
dtype: int64

In [6]:
# na 드롭
merged_df.dropna(inplace=True)


# 필요없는 열 제거
merged_df.drop(columns=['교육청'], inplace=True)
merged_df.drop(columns=['구분'], inplace = True)

In [7]:
def age(x):
    a = 0
    if x['학교급'] == '초등학교':
        a = 7
    elif x['학교급'] == '중학교':
        a = 13
    else:
        a = 16
    return a

In [8]:
def age2(y):
    b = 0
    if y['사고자학년'] == '1학년':
        b = y['추정나이'] + 0
    elif y['사고자학년'] == '2학년':
        b = y['추정나이'] + 1
    elif y['사고자학년'] == '3학년':
        b = y['추정나이'] + 2
    elif y['사고자학년'] == '4학년':
        b = y['추정나이'] + 3
    elif y['사고자학년'] == '5학년':
        b = y['추정나이'] + 4
    elif y['사고자학년'] == '6학년':
        b = y['추정나이'] + 5
    return b

In [11]:
merged_df['추정나이'] = merged_df.apply(age, axis=1)
merged_df['추정나이'] = merged_df.apply(age2, axis=1)

In [15]:
col_to_sum = ['요양급여', '장해급여', '간병급여', '유족급여', '장례비', '위로금', '보전비용']

merged_df['보상급여'] = merged_df[col_to_sum].sum(axis=1)
merged_df.drop(columns=col_to_sum, inplace=True)

In [17]:
merged_df

,지역,학교급,사고자구분,사고자학년,사고자성별,사고시간,사고장소,사고부위,사고형태,사고당시활동,추정나이,보상급여
0,전남,고등학교,일반학생,2학년,남,쉬는시간,강당(체육관),골반/엉덩이,그밖의 손상 사고,기타,17,31900
1,대구,초등학교,일반학생,6학년,여,체육,운동장,두피,그밖의 손상 사고,기타 스포츠 활동,12,53000
2,경남,고등학교,일반학생,1학년,남,그 밖의 교육활동 시간,기타 교외,기타,그밖의 손상 사고,기타,16,15027050
3,전북,초등학교,일반학생,5학년,남,쉬는시간,일반(교과)교실,어깨,그밖의 손상 사고,기타,11,3114900
4,제주,초등학교,일반학생,2학년,남,식사시간(간식 포함),기타 교외,치아,고정된 물체와의 부딪힘,기타,8,1106500
...,...,...,...,...,...,...,...,...,...,...,...,...
425209,서울,중학교,일반학생,3학년,여,체육,강당(체육관),손가락,고정된 물체와의 부딪힘,피구,15,97200
425210,서울,초등학교,일반학생,5학년,남,체육,강당(체육관),복합부위,사람과의 부딪힘,기타 구기,11,585950
425211,서울,기타학교,일반학생,3학년,여,현장학습,기타 교외,무릎,넘어짐,"걷기/뛰기, 오르내리기",18,0
425212,서울,기타학교,일반학생,2학년,여,쉬는시간,계단,어깨,넘어짐,"걷기/뛰기, 오르내리기",17,284910


In [18]:
merged_df.to_csv('../data/Compensation_pre.csv')